In [9]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0)

In [15]:
QA = """JEFFREY JOHN ZEKAUSKAS: Okay. And then quickly, I realize that your volumes in the United States year-over-year or flat, but if you had or I'm sorry in the Americas, but if you had to break it up into package, merchant and on-site, and compare the volume changes, what would they look like for the second quarter?"""

ARTICLE = """
Great question, Noel. I think the Ameresco project we're doing is just an example of our platform's capability to take advantage of what is a growing movement to leverage anaerobic digesting at wastewater treatment facilities and then to utilize that fuel as a way power. And we have demonstrated our capabilities to do that on a number of wastewater projects. And one of the core benefits that we have there is our ability to directly utilize that fuel. In other words, we don't need to upgrade that fuel to pipeline quality. So that fuel never leaves the premise, if you will.

In addition to that, again, just like I just talked about absorption chilling and the benefit to leverage our thermal energy, we're able to use that thermal energy to feed back into the anaerobic digester to accelerate that process.

So the other thing that I would say there that I would just want to point out, and I mentioned in my prepared remarks, with the solution that we're doing with Ameresco, there's two other opportunities there that we could potentially leverage our technology take advantage of, and that's the production of hydrogen, similar to what we're doing at Toyota. And then also, these modules that are being deployed are carbon recovery capable. And so, to the degree that we can utilize that carbon or there's a need for that, whether it be for a product or sequestration or otherwise, that's just another value stream that can be created there.

And so, along with the Ameresco project, that's near-term revenue opportunity for us and we see that as a market that will continue to grow.

"""

#print(summarizer(QA, max_length=130, min_length=30, do_sample=False))
print(summarizer(ARTICLE, max_length=200, min_length=80, do_sample=False))

[{'summary_text': "The Ameresco project is just an example of our platform's capability to take advantage of what is a growing movement to leverage anaerobic digesting at wastewater treatment facilities. And we have demonstrated our capabilities to do that on a number of wastewater projects. And one of the core benefits that we have there is our ability to directly utilize that fuel. So that fuel never leaves the premise, if you will."}]


In [1]:
import os
import re
import pandas as pd
import numpy as np
from transformers import pipeline
from docx import Document
import spacy

summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0)

# Read the files inside the folder
def read_files(folder):
    files = os.listdir(folder)
    files = [f for f in files]
    return files

folder = 'UniNexis'
files = read_files(folder)
i=0
for file in files:
    i = i+1
    print(i, file)
    

1 Advent Technologies Holdings, Inc. ( ADN ) Q4 2023 Earnings Call Transcript.docx
2 Aemetis, Inc. ( AMTX ) Q2 2024 Earnings Call Transcript.docx
3 African Rainbow Minerals Limited (AFBOF) Q4 2024 Earnings Call Transcript.docx
4 Air Products and Chemicals, Inc. (APD) Jefferies Industrials Conference - (Transcript).docx
5 Air Products and Chemicals, Inc. ( APD ) Q2 2024 Earnings Call Transcript.docx
6 Air Products and Chemicals, Inc. ( APD ) Q3 2024 Earnings Call Transcript.docx
7 Arq, Inc. ( ARQ ) Q2 2024 Earnings Call Transcript.docx
8 ATCO Ltd. ( ACLLF ) Q2 2024 Earnings Call Transcript.docx
9 Babcock & Wilcox Enterprises Inc. (BW) Q2 2024 Earnings Call Transcript.docx
10 Baker Hughes Company (BKR) Barclays 38th Annual CEO Energy-Power Conference (Transcript).docx
11 Baker Hughes Company ( BKR ) Q2 2024 Earnings Call Transcript.docx
12 Ballard Power Systems Inc. ( BLDP ) Q2 2024 Earnings Call Transcript.docx
13 Bayerische Motoren Werke Aktiengesellschaft (BMWYY) Q2 2024 Earnings Call

In [4]:
nlp = spacy.load("en_core_web_sm")
text  = 'Morten Holum'
doc = nlp(text)
print(doc.ents)

()


In [5]:
def get_text(document, QA_clean, par_H2, file):
    # Replace the text in the document
    for para in document.paragraphs:
        if 'Subscribe to Seeking Alpha for more content like this' in para.text:
            para.text = para.text.replace('Subscribe to Seeking Alpha for more content like this', '')

    # Loop through the paragraphs to find the start of the Q&A session
    for idx, para in enumerate(document.paragraphs):
        # Get the title of the document
        if para.style.name == 'Heading 1':
            title = para.text

        # Get paragraphs that contain the Q&A session and Hydrogen from the presentation
        if 'question-and-answer' in para.text.lower():
            
            # Extract the speech from the document
            H2_par = document.paragraphs[:idx]
            try:
                for idH2, paraH2 in enumerate(H2_par):
                    if 'hydrogen' in paraH2.text.lower():
                        previous_par    = [document.paragraphs[idH2-1] if document.paragraphs[idH2-1].text != '' else document.paragraphs[idH2-2]][0].text
                        next_par        = [document.paragraphs[idH2+1] if document.paragraphs[idH2+1].text != '' else document.paragraphs[idH2+2]][0].text

                        par_H2.append([file, title, previous_par, paraH2.text, next_par])
            except Exception as e:
                print('Error in extracting Hydrogen paragraphs:', e)
                continue
            
            # Extract the Q&A session from the document
            QAs = document.paragraphs[idx + 1:]
            QAs = [para.text for para in QAs]
            id_0 = 0
            speaker = ''
            for id_x, para in enumerate(QAs):
                try:
                    if len(para.split()) < 5:
                        doc = nlp(para)

                        if id_x-id_0 > 1:
                            QA_txt = ' '.join(QAs[id_0+1:id_x])
                            QA_sec = {'File_Name': file, 'Title': title, 'Speaker': speaker, 'Text': QA_txt}
                            QA_clean.append(QA_sec)
                            id_0 = id_x 
                        else:
                            id_0 = id_x

                        for token in doc:
                            if token.pos_ == 'PROPN' and token.dep_ == 'compound':
                                speaker = token.text + ' ' + token.head.text 
                except Exception as e:
                    print('Error in extracting Q&A paragraphs:', e)
                    continue    
            
    return QA_clean, par_H2

def read_files(folder):
    # Filter only .docx files in the folder
    return [file for file in os.listdir(folder) if file.endswith('.docx')]

In [6]:
folder = 'UniNexis'
files = read_files(folder)

# Initialize empty list to store Q&A text
QA_clean = []
par_H2 = []
results = []
i=0
H2_0 = 0
QA_0 = 0

# Loop through the files to extract the Q&A text
for file in files:

    # Load the document
    file_path = os.path.join(folder, file)
    doc = Document(file_path)

    QA_clean, par_H2 = get_text(doc, QA_clean, par_H2, file)
    r_extracted_=[i, file, len(par_H2)-H2_0, len(QA_clean)-QA_0]
    results.append(r_extracted_)
    
    i = i+1
    H2_0 = len(par_H2)
    QA_0 = len(QA_clean)

# Create a dataframe from the Q&A text
QA_df       = pd.DataFrame(QA_clean)
H2_df       = pd.DataFrame(par_H2, columns=['file_name', 'title', 'Previous_par', 'Hydrogen_par', 'Next_par'])
results_df  = pd.DataFrame(results, columns=['ID', 'File_Name', 'Hydrogen', 'QA'])

In [200]:

def is_question_dep(text):
    # Extract sentences
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    questions = [sent for sent in sentences if '?' in sent]
    return questions

QA_df['is_Question'] = QA_df['Text'].apply(is_question_dep)
H2_df['para_H2'] = H2_df.apply(lambda x: x['Previous_par'] + ' ' + x['Hydrogen_par'] + ' ' + x['Next_par'], axis=1)



In [12]:
# Import tools for clustering
import umap.umap_ as umap
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import KeyBERTInspired
from bertopic.representation import MaximalMarginalRelevance
from bertopic.representation import PartOfSpeech
import hdbscan
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.metrics import silhouette_samples
from sklearn.decomposition import PCA
# Import transformers and others
from transformers import pipeline
from sentence_transformers import SentenceTransformer, SimilarityFunction, util
from transformers import AutoTokenizer, AutoModelForTokenClassification
# Import sk
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
#================================================================================================
# Function to rescale the embeddings (fromg BERTopic documentation)
#================================================================================================

def rescale(x, inplace=False):
  """ Rescale an embedding so optimization will not have convergence issues.
  """
  if not inplace:
      x = np.array(x, copy=True)
  x /= np.std(x[:, 0]) * 10000

  return x

#================================================================================================
# Define BERTopic model function
#================================================================================================

def BERTopic_model(docs, embeddings, n_comp=20, n_neigh=25, minS_cluster=5):
  # Initialize and rescale PCA embeddings
  pca_embeddings = rescale(PCA(n_components=n_comp).fit_transform(embeddings))

  # Embedding model
  embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
  #embedding_model = None

  # Reduce dimensionality
  umap_model      = umap.UMAP(n_neighbors=n_neigh, 
                              n_components=n_comp, 
                              metric='cosine',
                              random_state=42,
                              init=pca_embeddings,
                              )

  # Cluster the reduced data
  hdbscan_model   = hdbscan.HDBSCAN(min_cluster_size=minS_cluster, 
                                      metric='euclidean', 
                                      cluster_selection_method='eom',
                                      prediction_data = True)

  # Vectorizer
  vectorizer_model = CountVectorizer(stop_words="english",
                                      ngram_range=(1, 2),
                                      )

  # Topic representation
  ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

  # Representation model
  representation_model = KeyBERTInspired()
  #representation_model = MaximalMarginalRelevance(diversity=0.3)

  # All steps together
  topic_model = BERTopic(
    embedding_model=embedding_model,          # Step 1 - Extract embeddings
    umap_model=umap_model,                    # Step 2 - Reduce dimensionality
    hdbscan_model=hdbscan_model,              # Step 3 - Cluster reduced embeddings
    vectorizer_model=vectorizer_model,        # Step 4 - Tokenize topics
    ctfidf_model=ctfidf_model,                # Step 5 - Extract topic words
    representation_model=representation_model,# Step 6 - (Optional) Fine-tune topic representations
    calculate_probabilities=True
  )

  # Fit the BERTopic model
  topics, probs = topic_model.fit_transform(docs,  embeddings)

  return topics, probs, topic_model

#================================================================================================
# Function to calculate silhouette score of the clustering
#================================================================================================

def silhouette_score_calc(embeddings, topics):
  # Print the silhouette score
  non_noise_indices   = [i for i, topic in enumerate(topics) if topic != -1]
  filtered_embeddings = embeddings[non_noise_indices]
  filtered_topics     = [topics[i] for i in non_noise_indices]

  silhouette_vals     = silhouette_samples(filtered_embeddings, filtered_topics)
  silhouette_avg      = silhouette_score(filtered_embeddings, filtered_topics)

  # Create a DataFrame to hold the results
  results = pd.DataFrame({'topic': filtered_topics, 'silhouette_score': silhouette_vals})

  # Group by topic and calculate the average Silhouette score per topic
  avg_silhouette_per_topic = results.groupby('topic')['silhouette_score'].mean().reset_index()

  # Count how many topics are negative
  negative_topics = avg_silhouette_per_topic[avg_silhouette_per_topic['silhouette_score'] < 0].shape[0]

  return silhouette_avg, negative_topics

In [15]:
# Embedding model using the specter model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2", device='cuda')

# Create embedding of abstracts
#abstracts = H2_df.melt(id_vars=['file_name'], var_name='paragraphs', value_name='value')
abstracts = H2_df['Hydrogen_par']
# clear duplicates
#abstracts = abstracts.drop_duplicates(subset='value')
#abstracts['value'] = abstracts['value'].apply(lambda x: x if len(x.split()) > 50 else None)
#abstracts = abstracts.dropna(subset=['value'])
#abstracts = abstracts.reset_index(drop=True)
#abstracts = abstracts['value']

embeddings_MiniLM = embedding_model.encode(abstracts, show_progress_bar=True)

print(len(abstracts), len(embeddings_MiniLM))

Batches:   0%|          | 0/39 [00:00<?, ?it/s]

1235 1235


In [16]:
# Embeddings to use
embedings_x     = embeddings_MiniLM

# Selected parameters
## 30, 50, 50
## 20, 30, 40
## 20, 60, 50
_minCluster = 20
_components = 60
_neighbors  = 50

# Fit the BERTopic model
topics, probs, topic_model = BERTopic_model(
                                abstracts,
                                embedings_x, 
                                n_comp        = _components, 
                                n_neigh       = _neighbors, 
                                minS_cluster  = _minCluster,
                                )
silhouette_avg, negative_topics = silhouette_score_calc(embedings_x, topics)
print(f"Silhouette score: {silhouette_avg}")

# Reduce embeddings to 2D for visualization
reduced_embeddings = umap.UMAP( n_neighbors=_neighbors, n_components=2, 
                          min_dist=0.0, metric='cosine').fit_transform(embedings_x)

# Get the abstracts information from the topic model
document_info = topic_model.get_document_info(abstracts)

# Visualize the topics
topic_model.visualize_documents(abstracts, reduced_embeddings=reduced_embeddings, 
                                hide_document_hover=True, hide_annotations=True)

Silhouette score: 0.06804703176021576


In [19]:
def flexible_summarizer(text):
    length = len(text.split())
    
    if length < 30:  # Skip summarizing if the text is too short
        return []
    else:
        max_len = min(150, int(0.5 * length))  # Dynamic max length as a fraction of the text length
        min_len = min(50, int(0.3 * length))   # Dynamic min length
        return length, max_len, min_len, summarizer(text, max_length=max_len, min_length=min_len, do_sample=False)

#H2_df['para_H2'] = H2_df.apply(lambda x: x['Previous_par'] + ' ' + x['Hydrogen_par'] + ' ' + x['Next_par'], axis=1)

#H2_df['resumen'] = H2_df['par_H2'].apply(lambda x: flexible_summarizer(x))

representation = document_info[['Document', 'Topic', 'Representative_Docs']]
representation = representation[representation['Topic'] != -1]
representation['summary'] = representation['Document'].apply(lambda x: flexible_summarizer(x))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [20]:
# save the results
representation.to_excel('representation.xlsx', index=False)

In [120]:


def flexible_summarizer(text):
    length = len(text.split())
    
    if length < 30:  # Skip summarizing if the text is too short
        return []
    else:
        max_len = min(250, int(0.4 * length))  # Dynamic max length as a fraction of the text length
        min_len = min(60, int(0.2 * length))   # Dynamic min length
        return summarizer(text, max_length=max_len, min_length=min_len, do_sample=False)

QA_df['Summary'] = QA_df['Text'].apply(flexible_summarizer)


In [117]:

# Create a dataframe from the Q&A text
QA_section = QA_clean
QA_section = pd.DataFrame(QA_section, columns=['QA'])
QA_section = QA_section[QA_section['QA'] != '/split/']
QA_section = QA_section.reset_index(drop=True)

# Select the pairs of questions and answers
Questions = QA_section['QA'][QA_section.index % 2 == 0].reset_index(drop=True)
Answers = QA_section['QA'] [QA_section.index % 2 == 1].reset_index(drop=True)

QA_df = pd.DataFrame({'Questions': Questions, 'Answers': Answers})
#

print(QA_df)


                                            Questions  \
0   So if I'm reading the financials correctly and...   
1   Yes. And your math is right. It's just over $5...   
2   606. That from a contractual point of view, th...   
3   Yes. It's good. It's -- and Colin, I especiall...   
4   So again, on this question, right, as we've ta...   
5   And let me -- I'm going to -- I'm probably goi...   
6   So my second question is about the balance she...   
7   I just wanted to focus on the -- so the revenu...   
8   So I can tell you there's one customer where i...   
9   And when you think about liquefier Sanjay, it'...   
10  So first question, I'd like to ask is about th...   
11  Dean, congrats on the new role. Maybe this is ...   
12  Yes. I think it's a little too early to provid...   
13  So I've got a couple on Louisiana. First off, ...   
14  So Sherif, on that, so obviously, it's a 50-50...   
15  Yes, sure. Keep in mind, right, we've already ...   
16  Yes, I would just -- so jus